# Lecture 3: More Python

## Coding style

I won't bother you with my personal coding preferences ... instead, please refer to the official Python coding style guide available at:

https://www.python.org/dev/peps/pep-0008/

Generally speaking, a consistent coding style (naming, spacing, intendation, ...) is always encouraged as it makes your code more readable, improves maintainability as well as scalability.

## Arbitrary precision floats

By default, Python supports arbitrary length integers. For arbitrary precision float operations, we can use the ``mpmath`` library  which allows us to work with floats with a defined number of decimals:

In [ ]:
from mpmath import mp

mp.dps = 1024 # mp-floats will have 1024 digits

To create an mp-float object, just use the mpf method:

In [ ]:
mpFloat = mp.mpf(2)

print(type(mpFloat),mpFloat)

Basic operators will be overloaded to deal with mp-floats and regular floats will be converted if necessary:

In [ ]:
frac = 1/mp.mpf(3)

print("Type: ", type(frac))
print("Length: ", len(str(frac))) # Note that we have to typecast frac first before we can apply the len function
print(frac)

Furthermore, ``mpmath`` provides all basic mathematical constants and functions that we already know from the ``math`` library:

In [ ]:
print(mp.sqrt(2) + mp.exp(mp.pi)) # No typecast from float to mpfloat required ...

## Formatted output

For a quick look, unformatted output is just fine. But at times, we would like to have more control, e.g. regarding the number format or the display precision of floats. Python supports classic C style format specifiers and escape sequences:

In [ ]:
a = 1
b = 10

print("%1d divided by %2d is %1.6f" %(a,b,a/b))

Strings also provide the ``format`` method as an alternative that is more on line with the zen of Python: 

In [ ]:
import math

n = 10
i = 2
pi = math.pi

print("The first {0:2d} digits of the {1:1d}nd best number are: {2:1.10f}".format(n+1, i, pi))

The latest addition to the ways of producing formatted output in Python (_there should be one obvious way to do it ..._) are f-strings:

In [ ]:
a = 'allow'
b = 2

print(f"f-strings {a} {b} insert you variables into strings just where you need them. Smash!")

## String operations

In Python, strings just work like lists of characters:

In [ ]:
name = input("Enter your name: ") 
name = name.strip() # Remove leading and trailing whitespaces

print("\nHello "+name+"!")

Using the ``split`` method, we can pass a string at which the input string will be chopped into list elements:

In [ ]:
names = name.split(" ") # Split string into a list of substrings

print("First name:", names[0])
print("Last name:", names[-1])

print("Your name contains",len(name),"characters, including",name.count(" "),"whitespace(s)")

print("First character:", name[0])
print("Last character:", name[-1]) # -1 refers to the last item, -2 to the second last, ...

print("Reversed:", name[::-1])

print("Replace a by u:", name.replace("a", "u"))

Even numbers can be converted to strings which is an elegant way to calculate the cross-sum:

In [ ]:
primes = [1,2,3,5,7,11,13,17,19,23,29] # List of prime numbers

for prime in primes: # Iterate through the list

    csum = 0 # Cross-sum accumulation variable
    pstr = str(prime) # Convert prime to string
    
    for dig in pstr: # Iterate through every digit of
        csum += int(dig)
        
    print("The cross sum of",prime,"is",csum)

Lists of strings can be joined back to a single string. We only need to provide a joining character:

In [ ]:
stringList = ["I", "need", "more", "mate"] # A random list of strings

string = " ".join(stringList) # Turn the list of strings into a single string

print(stringList)
print(string.lower())
print(string.upper())
print(string.split()) # And back to a list

Strides and slicing work just like oridnary lists:

In [ ]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

print( alphabet[::2] ) # Stride: Every 2nd character
print( alphabet[:10:2] ) # Every 2nd char of the first ten elements

A few lines of code are all you need to calculate some basic text statistics such as the distribution of characters:

In [ ]:
alphabet = "abcdefghijklmnopqrstuvwxyz"
text = "This is some random text I have typed for illustrative purposes. Of course I could make it longer and longer but it quickly starts getting tiresome. "\
    "One more sentence and we are done with that. Just one more sentence. Cut the crap, I cannot take it anymore. "\
    "This is the final sentence of this never-ending string."

stat = []

print(text)
print("Some statistics:")

for char in alphabet:
    freq = text.count(char)
    stat.append( freq )
    print("Character",char,":", freq)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

idx = range(0, len(alphabet)) # Character indices
alphaList = list(alphabet) # Typecasting strings into lists results in a list of individual chars

plt.figure(figsize=(15,10)) # Create a figure
plt.vlines(idx, 0, stat, color='blue')
plt.xlabel('Letter')
plt.xticks(idx, labels=alphaList) # Instead of indices, show the list of characters as x ticks
plt.ylabel('N')
plt.show()

## File IO

At times, we need to store data to the harddrive of our computer. Say, we have created lists of x and y values of our favourite function $f(x) = x \, \exp \left( -x^2 \right)$:

In [ ]:
import math

def f(x):
    return x*math.exp(-x**2)

x = [ 0.01*xi for xi in range(500) ]
y = list(map(f,x))

Now we would like to store the results in a file named ``vals.txt``. Every line shall contain a x and a y value, separated by a blank space. For that, we create a file object in write mode and use the ``write`` method to write adequately formatted strings:

In [ ]:
file = open("vals.csv","w") # csv: Comma separated values

for vals in zip(x, y):
    # strLine = str(vals[0])+" "+str(vals[1])+"\n" # Traditional string construction with typecast
    strLine = f"{vals[0]},{vals[1]}\n" # Modern f-string approach
    file.write(strLine)
    
file.close()

If you like C-style loops, the next cell may float your boat:

In [ ]:
file = open("vals.csv","w")

for i in range(len(x)):
    strLine = "{},{}\n".format(x[i],y[i])
    file.write(strLine)
    
file.close()

On the other hand, by combining ```zip``` and ```join``` we can get rid of indices altogether and deal with an arbitrary number of columns. Using the ```with``` block, we can further drop the explicit file pointer close statement:

In [ ]:
# May look ugly for just two columns, but really shines once there are more ...
with open("vals.csv","w") as file:
    file.write("\n".join( str(line)[1:-1] for line in zip(x,y)) ) # String slicing removes tuple parentheses
    
file.close()

Now we go the other way: Let's get the data back from the harddisk to our RAM. The Python way is to iterate through every line of the file. Before we read in all data, let's see what we have here:

In [ ]:
del x
del y

file = open("vals.csv","r")

for line in file:
    print(line, type(line))
    break

file.close()

So we are dealing with a simple string that we need to parse back into lists. First, we apply the ``strip`` method to get rid of trailing and leading whitespaces as well as escape sequences. Then we use the ``split`` method to divide the string into a list of two entries, the first being column 1 and the second being column 2:

In [ ]:
x = []
y = []

file = open("vals.csv","r")

for line in file:
    line.strip() # Remove trailing and leading whitespaces as well as the newline char
    cols = line.split(",") # Split columns
    x.append( float(cols[0]) )
    y.append( float(cols[1]) )

file.close()

print(len(x), len(y))

Similarily, we can read entire files into a single string and process it afterwards:

In [ ]:
file = open("metaphysics.txt", "r") # open the file in read mode (r)

contents = file.read()

file.close()

What did the ``read`` method return?

In [ ]:
print(type(contents))
print(len(contents))

So we are dealing with an ordinary string of 6734 characters:

In [ ]:
print(contents)

## Dictionaries

When dealing with arrays of data, there are times when we do not really care about the index of some element and the exact order but would rather like to access elements using a string. A classic approach to that problem is to create two arrays which then share one index:

In [ ]:
cName = ["XBT", "ETH", "LTC", "USDT", "XRP"]
balance = [0.08, 0.04, 0.1, 51, 30]

for coin in zip(cName, balance):
    print("My wallet has",coin[1],coin[0])

While this may work out, the downside is that data that forms a unit (the name of the coin and its balance) is being separated. For cases like that, Python features the list-like ``dictionary`` object whose elements are accessed via strings rather than integers:

In [ ]:
wallet = { "XBT": 0.08, "ETH": 0.04, "LTC": 0.1, "USDT": 51, "XRP": 30}

wallet = {}

for coin in zip(cName, balance):
    wallet[coin[0]] = coin[1]

Note that integer-typed keys are still supported, you can even mix string keys and integer keys. New elements can be added at any time by assigning a value to a key:

In [ ]:
wallet["BCH"] = 0.05

for coin in wallet: # Iterate through all pairs of elements within the dictionary
    print("My wallet has",wallet[coin],coin)

Any kind of object can be added to a dictionary, even dictionaries. Such nested dictionaries behave just like nested lists and require a second set of brackets to access elements within the nested dictionary. To remove a certain element, use ``del`` followed by the name of the dictionary and the key:

In [ ]:
del wallet["BCH"]

print(wallet)

There are a couple of useful methods when dealing with dictionaries:

In [ ]:
print(dir(wallet))

``items`` returns a list of tuple of all key-element pairs within the dictionary, ``keys`` returns a list of all keys that are being used within the dictionary, ``pop`` takes a key, returns the associated element and removes it from the dictionary. Feel free to try them at home ...

One particular use cases of dictionaries is some sort of database to store and return information that would otherwise require lengthy conditional blocks:

In [ ]:
capital = {"Germany" : "Berlin", "Poland" : "Warsaw", "Czech Republic" : "Prague",\
           "Austria" : "Vienna", "Switzerland" : "Bern", "France" : "Paris",\
           "Luxembourg" : "Luxembourg", "Belgium" : "Bruxelles",\
           "Netherlands" : "Amsterdam", "Denmark" : "Copenhagen"}

for country in capital:
    print("The capital of",country,"is",capital[country])